### Read the data from the input Excel file

In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
inputFileName = "TransportationPb_Data.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
supplyDF = pd.read_excel(inputFileName, "Supply", skiprows=0, index_col=0)
demandDF = pd.read_excel(inputFileName, "Demand", skiprows=0, index_col=0)
freightDF = pd.read_excel(inputFileName, "Freight", skiprows=0, index_col=2)


### Set the data into dict, easier to manipulate

In [3]:
plants = supplyDF.to_dict('index')
markets = demandDF.to_dict('index')
   

freightCost = {}  #freightCost[iPlant][iMarket]
for index, row in freightDF.iterrows():
    if row['Plant'] not in freightCost : freightCost[row['Plant']] = {}
    freightCost[row['Plant']][row['Market']] = row['Cost']
    

In [4]:
# fct obj : min cost st demands are verfied
from pulp import *
prob = LpProblem("Transportation_problem",LpMinimize)

### Add variables


In [5]:
Xvar = LpVariable.dicts("X", ((i,j) for i in plants for j in markets), 
                             lowBound=0, cat='Continuous')
DemandVar= LpVariable.dicts("Demand", (i for i in markets), lowBound=0, cat='Continuous')
CapacityVar= LpVariable.dicts("Capacity", (i for i in plants), lowBound=0, cat='Continuous')


### Add objectives

In [6]:
prob += lpSum([(freightCost[i][j] + plants[i]['Cost'])* Xvar[i,j] for i in plants for j in markets ])

In [7]:
#Notre problème renferme 200 variables Xij, 20 contraintes Capa, 10 contraintes de demand

In [8]:
# for i in plants.keys():
#     for j in markets.keys():
#         print(Xvar[i,j])

### Add constraints

In [9]:
#Capacity constraint (for each plant)
for i in (plants) : 
        prob += lpSum(Xvar[i,j] for j in markets) <= plants[i]['Capacity'] , "capacity[%s]"%(i)



In [10]:
# # #Demand constraint
for j in (markets) :
    prob += lpSum(Xvar[i,j] for i in plants) >= markets[j]['Demand'] , "demand[%s]"%(j)

In [11]:
prob

Transportation_problem:
MINIMIZE
19*X_('P1',_'M1') + 22*X_('P1',_'M10') + 17*X_('P1',_'M2') + 18*X_('P1',_'M3') + 19*X_('P1',_'M4') + 22*X_('P1',_'M5') + 16*X_('P1',_'M6') + 17*X_('P1',_'M7') + 15*X_('P1',_'M8') + 16*X_('P1',_'M9') + 13*X_('P10',_'M1') + 16*X_('P10',_'M10') + 14*X_('P10',_'M2') + 12*X_('P10',_'M3') + 16*X_('P10',_'M4') + 18*X_('P10',_'M5') + 17*X_('P10',_'M6') + 14*X_('P10',_'M7') + 14*X_('P10',_'M8') + 18*X_('P10',_'M9') + 14*X_('P11',_'M1') + 10*X_('P11',_'M10') + 9*X_('P11',_'M2') + 13*X_('P11',_'M3') + 13*X_('P11',_'M4') + 10*X_('P11',_'M5') + 12*X_('P11',_'M6') + 14*X_('P11',_'M7') + 12*X_('P11',_'M8') + 7*X_('P11',_'M9') + 29*X_('P12',_'M1') + 26*X_('P12',_'M10') + 30*X_('P12',_'M2') + 32*X_('P12',_'M3') + 23*X_('P12',_'M4') + 28*X_('P12',_'M5') + 28*X_('P12',_'M6') + 27*X_('P12',_'M7') + 26*X_('P12',_'M8') + 26*X_('P12',_'M9') + 34*X_('P13',_'M1') + 32*X_('P13',_'M10') + 32*X_('P13',_'M2') + 31*X_('P13',_'M3') + 35*X_('P13',_'M4') + 33*X_('P13',_'M5') + 27*X_('P

In [12]:
#prob.writeLP("blenderProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "X" in v.name:
        print(v.name, "=", v.varValue)
        


Status: Optimal
Objective =  16847.0
X_('P1',_'M1') = 0.0
X_('P1',_'M10') = 0.0
X_('P1',_'M2') = 80.0
X_('P1',_'M3') = 0.0
X_('P1',_'M4') = 0.0
X_('P1',_'M5') = 0.0
X_('P1',_'M6') = 0.0
X_('P1',_'M7') = 0.0
X_('P1',_'M8') = 0.0
X_('P1',_'M9') = 0.0
X_('P10',_'M1') = 0.0
X_('P10',_'M10') = 0.0
X_('P10',_'M2') = 0.0
X_('P10',_'M3') = 2.0
X_('P10',_'M4') = 0.0
X_('P10',_'M5') = 0.0
X_('P10',_'M6') = 0.0
X_('P10',_'M7') = 0.0
X_('P10',_'M8') = 0.0
X_('P10',_'M9') = 0.0
X_('P11',_'M1') = 0.0
X_('P11',_'M10') = 0.0
X_('P11',_'M2') = 44.0
X_('P11',_'M3') = 0.0
X_('P11',_'M4') = 0.0
X_('P11',_'M5') = 0.0
X_('P11',_'M6') = 0.0
X_('P11',_'M7') = 0.0
X_('P11',_'M8') = 0.0
X_('P11',_'M9') = 12.0
X_('P12',_'M1') = 0.0
X_('P12',_'M10') = 0.0
X_('P12',_'M2') = 0.0
X_('P12',_'M3') = 0.0
X_('P12',_'M4') = 75.0
X_('P12',_'M5') = 0.0
X_('P12',_'M6') = 0.0
X_('P12',_'M7') = 0.0
X_('P12',_'M8') = 7.0
X_('P12',_'M9') = 8.0
X_('P13',_'M1') = 0.0
X_('P13',_'M10') = 0.0
X_('P13',_'M2') = 0.0
X_('P13',_'M3') = 

Problème Dual 

#Le nombre de variables du pb dual = nb de contraintes du primal

```
# Ce texte est au format code
```



In [13]:
for name, c in list(prob.constraints.items()):
     if "capacity" in name or "demand" in name:
        print(name, ":", c.pi, "\t\t", c.slack)

capacity_P1_ : -14.0 		 -0.0
capacity_P2_ : -21.0 		 -0.0
capacity_P3_ : 0.0 		 11.0
capacity_P4_ : 0.0 		 -0.0
capacity_P5_ : -10.0 		 -0.0
capacity_P6_ : -20.0 		 -0.0
capacity_P7_ : -23.0 		 -0.0
capacity_P8_ : -9.0 		 -0.0
capacity_P9_ : 0.0 		 53.0
capacity_P10_ : -18.0 		 -0.0
capacity_P11_ : -22.0 		 -0.0
capacity_P12_ : -3.0 		 -0.0
capacity_P13_ : -2.0 		 -0.0
capacity_P14_ : -7.0 		 -0.0
capacity_P15_ : 0.0 		 15.0
capacity_P16_ : -16.0 		 -0.0
capacity_P17_ : -6.0 		 -0.0
capacity_P18_ : -8.0 		 -0.0
capacity_P19_ : -7.0 		 -0.0
capacity_P20_ : -11.0 		 -0.0
demand_M1_ : 29.0 		 -0.0
demand_M2_ : 31.0 		 -0.0
demand_M3_ : 30.0 		 -0.0
demand_M4_ : 26.0 		 -0.0
demand_M5_ : 29.0 		 -0.0
demand_M6_ : 29.0 		 -0.0
demand_M7_ : 29.0 		 -0.0
demand_M8_ : 29.0 		 -0.0
demand_M9_ : 29.0 		 -0.0
demand_M10_ : 29.0 		 -0.0


In [17]:
# # #New constraint
for i in (plants) :
  for j in (markets) :
    prob += Xvar[i,j]  <= 0.5 * markets[j]['Demand'] , "NewCinstarints plant[%s] market[%s]"%(i,j)